In [11]:
import numpy as np

In [12]:
A = np.random.randn(5,5)
print A

[[ 0.41258404 -0.20542644 -1.23279582 -0.75888768 -0.90286927]
 [-0.4598616   0.63643978 -1.17175839  0.97815087 -0.03711415]
 [-0.82008072 -1.30232578 -0.09943544  0.59427989  0.22602939]
 [ 2.67057734 -0.18071991  0.33267293  0.00787444 -2.16068362]
 [-0.07347611  0.4124793   0.0714828   1.93687578 -0.59885202]]


In [13]:
T = np.diag(np.array([1000., 0.01, 0.1, 1000., 50.]))
print T

[[  1.00000000e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00]
 [  0.00000000e+00   1.00000000e-02   0.00000000e+00   0.00000000e+00
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e-01   0.00000000e+00
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+03
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    5.00000000e+01]]


In [14]:
B = np.dot(A, T)
print B

[[  4.12584038e+02  -2.05426437e-03  -1.23279582e-01  -7.58887677e+02
   -4.51434635e+01]
 [ -4.59861605e+02   6.36439784e-03  -1.17175839e-01   9.78150871e+02
   -1.85570732e+00]
 [ -8.20080720e+02  -1.30232578e-02  -9.94354420e-03   5.94279890e+02
    1.13014696e+01]
 [  2.67057734e+03  -1.80719909e-03   3.32672933e-02   7.87443517e+00
   -1.08034181e+02]
 [ -7.34761105e+01   4.12479297e-03   7.14827976e-03   1.93687578e+03
   -2.99426011e+01]]


In [19]:
x = np.random.randn(5)
print x

[ 0.51250861 -2.12276928 -0.95843872 -0.37544444 -1.06926797]


In [20]:
y = np.dot(B, x)
print y

[  544.76600557  -600.84129207  -655.4646314   1481.22689577  -732.84532575]


In [21]:
mu = 0.01
H1 = np.dot(B.T, B) + mu*np.identity(5)
x1 = np.linalg.solve(H1, np.dot(B.T, y))
print x1

[ 0.51241861  0.01389832 -0.39210257 -0.37547516 -1.07129425]


In [23]:
P = np.diag(np.array([1./np.max(B[0,:]), 1./np.max(B[1,:]), 1./np.max(B[2,:]), 1./np.max(B[3,:]), 1./np.max(B[4,:]), ]))
print P

[[ 0.00242375  0.          0.          0.          0.        ]
 [ 0.          0.00102234  0.          0.          0.        ]
 [ 0.          0.          0.00168271  0.          0.        ]
 [ 0.          0.          0.          0.00037445  0.        ]
 [ 0.          0.          0.          0.          0.0005163 ]]


In [32]:
PB = np.dot(P, B)
Py = np.dot(P.T, y)
H2 = np.dot(PB.T, PB) + mu*np.identity(5)
x2 = np.linalg.solve(H2, np.dot(B.T, Py))
print x2

[  795.53700021    -1.21699532     8.08638311   -39.7064926   1133.99611555]


In [33]:
print x2

[  795.53700021    -1.21699532     8.08638311   -39.7064926   1133.99611555]
